In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 20629 (delta 47), reused 37 (delta 15), pack-reused 20531
Receiving objects: 100% (20629/20629), 16.42 MiB | 2.42 MiB/s, done.
Resolving deltas: 100% (15063/15063), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728
Receiving objects: 100% (21728/21728), 8.51 Mi

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynbhttps://zoom.us/j/99710187430?pwd=SEJGSWlzYUdRcnI3Nlh6MzlENmRQUT09qu
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=10acea9b52e22ce985e481344e147009b5f993ef34f3f6edca62c50627a5293c
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [6]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [7]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [8]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [9]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [10]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#   class_df = train[class_lst]
    target_df = train[target]

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [11]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [12]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [13]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [14]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [15]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [16]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [17]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [18]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [19]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [20]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [21]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,1.5,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,1.5,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,1.6,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,1.6,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,1.6,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,1.5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,1.5,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,1.4,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,1.3,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,1.3,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [22]:
dataset7 = train_to_supervised(dataset6, n_days)
dataset7

,WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),WS(t-140),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),...,T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,1.5,69.08,-12.0,9.0,0.000000,1.000000,0.0,0.0,1.5,69.06,-12.0,9.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,...,-12.0,9.0,-0.608761,0.793353,0.0,0.0,0.4,79.09,-13.0,9.0,-0.500000,0.866025,0.0,0.0,0.7,79.08,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,0.0,0.0
145,1.5,69.06,-12.0,9.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,...,-13.0,9.0,-0.500000,0.866025,0.0,0.0,0.7,79.08,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,0.0,0.0
146,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,...,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0,0.0,0.0
147,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,9.0,0.793353,0.608761,0.0,0.0,...,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0,2.2,77.69,-13.0,9.0,0.258819,0.965926,0.0,0.0,0.0,0.0
148,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,9.0,0.793353,0.608761,0.0,0.0,1.3,74.62,-11.0,9.0,0.866025,0.500000,0.0,0.0,...,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0,2.2,77.69,-13.0,9.0,0.258819,0.965926,0.0,0.0,2.2,71.57,-13.0,9.0,0.382683,0.923880,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,1.0,84.85,-8.0,9.0,-0.608761,0.793353,0.0,0.0,1.2,82.26,-8.0,9.0,-0.500000,0.866025,0.0,0.0,1.3,82.25,-8.0,9.0,-0.382683,0.923880,0.0,0.0,1.5,81.14,-8.0,9.0,-0.258819,0.965926,0.0,0.0,1.6,81.12,-8.0,9.0,-0.130526,0.991445,0.0,0.0,...,0.0,9.0,-0.965926,0.258819,0.0,0.0,1.9,51.77,0.0,9.0,-0.923880,0.382683,0.0,0.0,2.1,53.88,0.0,9.0,-0.866025,0.500000,0.0,0.0,2.6,53.87,0.0,9.0,-0.793353,0.608761,0.0,0.0,3.1,55.98,-1.0,9.0,-

In [23]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),WS(t-140),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),...,WS(t-5),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,1.5,69.08,-12.0,9.0,0.000000,1.000000,0.0,0.0,1.5,69.06,-12.0,9.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,...,0.4,82.89,-12.0,9.0,-0.608761,0.793353,0.0,0.0,0.4,79.09,-13.0,9.0,-0.500000,0.866025,0.0,0.0,0.7,79.08,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0
145,1.5,69.06,-12.0,9.0,0.130526,0.991445,0.0,0.0,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,...,0.4,79.09,-13.0,9.0,-0.500000,0.866025,0.0,0.0,0.7,79.08,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0
146,1.6,71.78,-12.0,9.0,0.258819,0.965926,0.0,0.0,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,...,0.7,79.08,-13.0,9.0,-0.382683,0.923880,0.0,0.0,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0
147,1.6,71.75,-12.0,9.0,0.382683,0.923880,0.0,0.0,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,9.0,0.793353,0.608761,0.0,0.0,...,1.1,81.09,-13.0,9.0,-0.258819,0.965926,0.0,0.0,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0,2.2,77.69,-13.0,9.0,0.258819,0.965926,0.0,0.0
148,1.6,75.20,-12.0,9.0,0.500000,0.866025,0.0,0.0,1.5,69.29,-11.0,9.0,0.608761,0.793353,0.0,0.0,1.5,72.56,-11.0,9.0,0.707107,0.707107,0.0,0.0,1.4,72.55,-11.0,9.0,0.793353,0.608761,0.0,0.0,1.3,74.62,-11.0,9.0,0.866025,0.500000,0.0,0.0,...,1.4,81.08,-13.0,9.0,-0.130526,0.991445,0.0,0.0,1.8,77.76,-14.0,9.0,0.000000,1.000000,0.0,0.0,2.0,77.77,-13.0,9.0,0.130526,0.991445,0.0,0.0,2.2,77.69,-13.0,9.0,0.258819,0.965926,0.0,0.0,2.2,71.57,-13.0,9.0,0.382683,0.923880,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,1.0,84.85,-8.0,9.0,-0.608761,0.793353,0.0,0.0,1.2,82.26,-8.0,9.0,-0.500000,0.866025,0.0,0.0,1.3,82.25,-8.0,9.0,-0.382683,0.923880,0.0,0.0,1.5,81.14,-8.0,9.0,-0.258819,0.965926,0.0,0.0,1.6,81.12,-8.0,9.0,-0.130526,0.991445,0.0,0.0,...,1.7,51.78,0.0,9.0,-0.965926,0.258819,0.0,0.0,1.9,51.77,0.0,9.0,-0.923880,0.382683,0.0,0.0,2.1,53.88,0.0,9.0,-0.866025,0.500000,0.0,0.0,2.6,53.87,0.0,9.0,-0.793353,0.608761,0.0,0.0,3.1,55.98,-1

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [24]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [25]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [26]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [27]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [28]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [29]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [30]:
testset6 = testset5[['WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
testset6.head(10)

,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,2.8,70.34,-6.1,9.5,0.000000,1.000000,0.0,0.0
1,2.9,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0
2,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0
3,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0
4,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0
5,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0
6,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0
7,2.9,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0
8,2.9,68.56,-9.1,9.5,0.866025,0.500000,0.0,0.0
9,2.7,70.75,-9.5,9.5,0.923880,0.382683,0.0,0.0


In [31]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),WS(t-140),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),...,WS(t-5),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,2.8,70.34,-6.1,9.5,0.000000,1.000000,0.0,0.0,2.9,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0,...,4.0,49.56,-4.5,9.5,-0.608761,0.793353,0.0,0.0,4.0,52.07,-4.5,9.5,-0.500000,0.866025,0.0,0.0,3.9,52.01,-4.5,9.5,-0.382683,0.923880,0.0,0.0,3.9,53.72,-4.4,9.5,-0.258819,0.965926,0.0,0.0,3.9,53.31,-4.3,9.5,-0.130526,0.991445,0.0,0.0
145,2.9,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0,...,4.0,52.07,-4.5,9.5,-0.500000,0.866025,0.0,0.0,3.9,52.01,-4.5,9.5,-0.382683,0.923880,0.0,0.0,3.9,53.72,-4.4,9.5,-0.258819,0.965926,0.0,0.0,3.9,53.31,-4.3,9.5,-0.130526,0.991445,0.0,0.0,3.9,54.93,-4.2,9.5,0.000000,1.000000,0.0,0.0
146,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0,...,3.9,52.01,-4.5,9.5,-0.382683,0.923880,0.0,0.0,3.9,53.72,-4.4,9.5,-0.258819,0.965926,0.0,0.0,3.9,53.31,-4.3,9.5,-0.130526,0.991445,0.0,0.0,3.9,54.93,-4.2,9.5,0.000000,1.000000,0.0,0.0,3.8,54.52,-4.1,9.5,0.130526,0.991445,0.0,0.0
147,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0,2.9,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0,...,3.9,53.72,-4.4,9.5,-0.258819,0.965926,0.0,0.0,3.9,53.31,-4.3,9.5,-0.130526,0.991445,0.0,0.0,3.9,54.93,-4.2,9.5,0.000000,1.000000,0.0,0.0,3.8,54.52,-4.1,9.5,0.130526,0.991445,0.0,0.0,3.7,56.81,-4.0,9.5,0.258819,0.965926,0.0,0.0
148,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0,2.9,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0,2.9,68.56,-9.1,9.5,0.866025,0.500000,0.0,0.0,...,3.9,53.31,-4.3,9.5,-0.130526,0.991445,0.0,0.0,3.9,54.93,-4.2,9.5,0.000000,1.000000,0.0,0.0,3.8,54.52,-4.1,9.5,0.130526,0.991445,0.0,0.0,3.7,56.81,-4.0,9.5,0.258819,0.965926,0.0,0.0,3.6,55.90,-3.8,9.5,0.382683,0.923880,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.6,60.79,-3.5,9.5,-0.608761,0.793353,0.0,0.0,0.6,60.30,-3.8,9.5,-0.500000,0.866025,0.0,0.0,0.6,61.60,-4.1,9.5,-0.382683,0.923880,0.0,0.0,0.6,61.43,-4.4,9.5,-0.258819,0.965926,0.0,0.0,0.6,62.83,-4.7,9.5,-0.130526,0.991445,0.0,0.0,...,1.5,41.92,-4.9,9.5,-0.965926,0.258819,0.0,0.0,1.5,42.88,-5.2,9.5,-0.923880,0.382683,0.0,0.0,1.6,45.58,-5.4,9.5,-0.866025,0.500000,0.0,0.0,1.7,45.58,-5.4,9.5,-0.793353,0.608761,0.0,0.0,1.8,44.37,-5.4,9.5,-0.707107,0.707107,0.0,0.0
332,0.6,60.30,

In [32]:
#testset8 = encoding(testset7)
#testset8.drop('season', axis = 1, inplace = True)
#testset8

In [33]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?
testset9

,WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),WS(t-140),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),...,WS(t-5),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,3.9,54.93,-4.2,9.5,0.000000e+00,1.000000e+00,0.000000,0.000000,3.8,54.52,-4.1,9.5,1.305262e-01,9.914449e-01,0.000000,0.000000,3.7,56.81,-4.0,9.5,2.588190e-01,9.659258e-01,0.000000,0.000000,3.6,55.90,-3.8,9.5,3.826834e-01,9.238795e-01,0.000000,0.000000,3.4,58.48,-3.7,9.5,5.000000e-01,8.660254e-01,0.000000,0.000000,...,1.8,58.17,-11.4,9.5,-6.087614e-01,7.933533e-01,0.000000,0.000000,1.8,57.15,-11.3,9.5,-5.000000e-01,8.660254e-01,0.000000,0.000000,1.8,57.15,-11.3,9.5,-3.826834e-01,9.238795e-01,0.000000,0.000000,1.8,55.81,-11.2,9.5,-2.588190e-01,9.659258e-01,0.000000,0.000000,1.7,54.92,-11.0,9.5,-1.305262e-01,9.914449e-01,0.000000,0.000000
289,3.8,54.52,-4.1,9.5,1.305262e-01,9.914449e-01,0.000000,0.000000,3.7,56.81,-4.0,9.5,2.588190e-01,9.659258e-01,0.000000,0.000000,3.6,55.90,-3.8,9.5,3.826834e-01,9.238795e-01,0.000000,0.000000,3.4,58.48,-3.7,9.5,5.000000e-01,8.660254e-01,0.000000,0.000000,3.2,58.05,-3.6,9.5,6.087614e-01,7.933533e-01,0.000000,0.000000,...,1.8,57.15,-11.3,9.5,-5.000000e-01,8.660254e-01,0.000000,0.000000,1.8,57.15,-11.3,9.5,-3.826834e-01,9.238795e-01,0.000000,0.000000,1.8,55.81,-11.2,9.5,-2.588190e-01,9.659258e-01,0.000000,0.000000,1.7,54.92,-11.0,9.5,-1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,52.40,-10.7,9.5,0.000000e+00,1.000000e+00,0.000000,0.000000
290,3.7,56.81,-4.0,9.5,2.588190e-01,9.659258e-01,0.000000,0.000000,3.6,55.90,-3.8,9.5,3.826834e-01,9.238795e-01,0.000000,0.000000,3.4,58.48,-3.7,9.5,5.000000e-01,8.660254e-01,0.000000,0.000000,3.2,58.05,-3.6,9.5,6.087614e-01,7.933533e-01,0.000000,0.000000,3.0,60.69,-3.6,9.5,7.071068e-01,7.071068e-01,0.000000,0.000000,...,1.8,57.15,-11.3,9.5,-3.826834e-01,9.238795e-01,0.000000,0.000000,1.8,55.81,-11.2,9.5,-2.588190e-01,9.659258e-01,0.000000,0.000000,1.7,54.92,-11.0,9.5,-1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,52.40,-10.7,9.5,0.000000e+00,1.000000e+00,0.000000,0.000000,1.7,51.58,-10.5,9.5,1.305262e-01,9.914449e-01,0.000000,0.000000
291,3.6,55.90,-3.8,9.5,3.826834e-01,9.238795e-01,0.000000,0.000000,3.4,58.48,-3.7,9.5,5.000000e-01,8.660254e-01,0.000000,0.000000,3.2,58.05,-3.6,9.5,6.087614e-01,7.933533e-01,0.000000,0.000000,3.0,60.69,-3.6,9.5,7.071068e-01,7.071068e-01,0.000000,0.000000,3.0,60.24,-3.5,9.5,7.933533e-01,6.087614e-01,0.000000,0.000000,...,1.8,55.81,-11.2,9.5,-2.588190e-01,9.659258e-01,0.000000,0.000000,1.7,54.92,-11.0,9.5,-1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,52.40,-10.7,9.5,0.000000e+00,1.000000e+00,0.000000,0.000000,1.7,51.58,-10.5,9.5,1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,49.37,-10.3,9.5,2.588190e-01,9.659258e-01,0.000000,0.000000
292,3.4,58.48,-3.7,9.5,5.000000e-01,8.660254e-01,0.000000,0.000000,3.2,58.05,-3.6,9.5,6.087614e-01,7.933533e-01,0.000000,0.000000,3.0,60.69,-3.6,9.5,7.071068e-01,7.071068e-01,0.000000,0.000000,3.0,60.24,-3.5,9.5,7.933533e-01,6.087614e-01,0.000000,0.000000,2.9,61.65,-3.5,9.5,8.660254e-01,5.000000e-01,0.000000,0.000000,...,1.7,54.92,-11.0,9.5,-1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,52.40,-10.7,9.5,0.000000e+00,1.000000e+00,0.000000,0.000000,1.7,51.58,-10.5,9.5,1.305262e-01,9.914449e-01,0.000000,0.000000,1.7,49.37,-10

In [34]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] #'DHI', 'DNI', 'season'
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),WS(t-140),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),...,WS(t-5),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,1.1,59.09,0.1,9.5,0.000000,1.000000,0.0,0.0,1.2,61.20,-0.4,9.5,0.130526,0.991445,0.0,0.0,1.3,65.76,-0.8,9.5,0.258819,0.965926,0.0,0.0,1.4,66.24,-0.9,9.5,0.382683,0.923880,0.0,0.0,1.5,69.70,-1.0,9.5,0.500000,0.866025,0.0,0.0,...,0.5,82.74,-2.6,9.5,-0.608761,0.793353,0.000000,0.000000,0.5,82.00,-2.6,9.5,-0.500000,0.866025,0.000000,0.000000,0.6,82.00,-2.6,9.5,-0.382683,0.923880,0.0,0.0,0.7,81.38,-2.6,9.5,-0.258819,0.965926,0.0,0.0,0.8,81.99,-2.7,9.5,-0.130526,0.991445,0.0,0.0
289,1.2,61.20,-0.4,9.5,0.130526,0.991445,0.0,0.0,1.3,65.76,-0.8,9.5,0.258819,0.965926,0.0,0.0,1.4,66.24,-0.9,9.5,0.382683,0.923880,0.0,0.0,1.5,69.70,-1.0,9.5,0.500000,0.866025,0.0,0.0,1.6,69.70,-1.0,9.5,0.608761,0.793353,0.0,0.0,...,0.5,82.00,-2.6,9.5,-0.500000,0.866025,0.000000,0.000000,0.6,82.00,-2.6,9.5,-0.382683,0.923880,0.000000,0.000000,0.7,81.38,-2.6,9.5,-0.258819,0.965926,0.0,0.0,0.8,81.99,-2.7,9.5,-0.130526,0.991445,0.0,0.0,0.8,80.92,-2.8,9.5,0.000000,1.000000,0.0,0.0
290,1.3,65.76,-0.8,9.5,0.258819,0.965926,0.0,0.0,1.4,66.24,-0.9,9.5,0.382683,0.923880,0.0,0.0,1.5,69.70,-1.0,9.5,0.500000,0.866025,0.0,0.0,1.6,69.70,-1.0,9.5,0.608761,0.793353,0.0,0.0,1.7,68.17,-1.0,9.5,0.707107,0.707107,0.0,0.0,...,0.6,82.00,-2.6,9.5,-0.382683,0.923880,0.000000,0.000000,0.7,81.38,-2.6,9.5,-0.258819,0.965926,0.000000,0.000000,0.8,81.99,-2.7,9.5,-0.130526,0.991445,0.0,0.0,0.8,80.92,-2.8,9.5,0.000000,1.000000,0.0,0.0,0.9,81.53,-2.9,9.5,0.130526,0.991445,0.0,0.0
291,1.4,66.24,-0.9,9.5,0.382683,0.923880,0.0,0.0,1.5,69.70,-1.0,9.5,0.500000,0.866025,0.0,0.0,1.6,69.70,-1.0,9.5,0.608761,0.793353,0.0,0.0,1.7,68.17,-1.0,9.5,0.707107,0.707107,0.0,0.0,1.8,67.68,-0.9,9.5,0.793353,0.608761,0.0,0.0,...,0.7,81.38,-2.6,9.5,-0.258819,0.965926,0.000000,0.000000,0.8,81.99,-2.7,9.5,-0.130526,0.991445,0.000000,0.000000,0.8,80.92,-2.8,9.5,0.000000,1.000000,0.0,0.0,0.9,81.53,-2.9,9.5,0.130526,0.991445,0.0,0.0,1.0,79.91,-3.0,9.5,0.258819,0.965926,0.0,0.0
292,1.5,69.70,-1.0,9.5,0.500000,0.866025,0.0,0.0,1.6,69.70,-1.0,9.5,0.608761,0.793353,0.0,0.0,1.7,68.17,-1.0,9.5,0.707107,0.707107,0.0,0.0,1.8,67.68,-0.9,9.5,0.793353,0.608761,0.0,0.0,2.0,64.30,-0.7,9.5,0.866025,0.500000,0.0,0.0,...,0.8,81.99,-2.7,9.5,-0.130526,0.991445,0.000000,0.000000,0.8,80.92,-2.8,9.5,0.000000,1.000000,0.000000,0.000000,0.9,81.53,-2.9,9.5,0.130526,0.991445,0.0,0.0,1.0,79.91,-3.0,9.5,0.258819,0.965926,0.0,0.0,0.9,79.91,-3.0,9.5,0.382683,0.923880,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,1.1,79.32,12.2,15.0,-0.608761,0.793353,0.0,0.0,1.2,80.41,11.7,15.0,-0.500000,0.866025,0.0,0.0,1.1,83.10,11.2,15.0,-0.382683,0.923880,0.0,0.0,1.1,84.37,10.8,15.0,-0.258819,0.965926,0.0,0.0,1.1,85.50,10.6,15.0,-0.130526,0.991445,0.0,0.0,...,1.0,53.81,16.4,15.5,-0.965926,0.258819,46.762086,2.251943,0.9,55.62,15.9,15.5,-0.923880,0.382683,1.466729,0.093831,0.8,57.52,15.3,15.5,-0.86

In [35]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

k = 0.03
quantiles = [0.1-k, 0.2-k, 0.3-k, 0.4-k, 0.5, 0.6-k, 0.7-k, 0.8-k, 0.9-k]


In [36]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.005, save_binary = True, subsample=0.7, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [37]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [38]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

0.07
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.04922
[1000]	valid_0's quantile: 1.02494
[1500]	valid_0's quantile: 1.01482
[2000]	valid_0's quantile: 1.00885
[2500]	valid_0's quantile: 1.00549
[3000]	valid_0's quantile: 1.00436
[3500]	valid_0's quantile: 1.00243
[4000]	valid_0's quantile: 1.00043
[4500]	valid_0's quantile: 0.99926
[5000]	valid_0's quantile: 0.998625
[5500]	valid_0's quantile: 0.997551
[6000]	valid_0's quantile: 0.996408
[6500]	valid_0's quantile: 0.994955
[7000]	valid_0's quantile: 0.994219
[7500]	valid_0's quantile: 0.993909
[8000]	valid_0's quantile: 0.993255
[8500]	valid_0's quantile: 0.992857
[9000]	valid_0's quantile: 0.992486
[9500]	valid_0's quantile: 0.992257
[10000]	valid_0's quantile: 0.992094
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 0.992094
0.17
[LightGBM] [War

,0.07,0.17,0.27,0.37,0.50,0.57,0.67,0.77,0.87
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [39]:
# Target2

models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
tock = time.time()
results_2.sort_index()

0.07
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.06036
[1000]	valid_0's quantile: 1.03999
[1500]	valid_0's quantile: 1.03232
[2000]	valid_0's quantile: 1.0269
[2500]	valid_0's quantile: 1.0231
[3000]	valid_0's quantile: 1.02136
[3500]	valid_0's quantile: 1.02002
[4000]	valid_0's quantile: 1.01818
[4500]	valid_0's quantile: 1.01848
Early stopping, best iteration is:
[4053]	valid_0's quantile: 1.01796
0.17
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.11147
[1000]	valid_0's quantile: 2.00028
[1500]	valid_0's quantile: 1.95938
[2000]	valid_0's quantile: 1.93379
[2500]	valid_0's quantile: 1.91733
[3000]	valid_0's quantile: 1.90431
[3500]	valid_0's quantile: 1.89029
[4000]

,0.07,0.17,0.27,0.37,0.50,0.57,0.67,0.77,0.87
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 6371.570283651352
(3888, 9) (3888, 9)


In [41]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [42]:
submission3.to_csv('submission7.csv')
!cp submission7.csv "drive/My Drive/"